**Quantlib를 통한 블랙솔즈 옵션 모형 구현**


In [2]:
pip install quantlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 24.5 MB/s eta 0:00:00


In [3]:
import QuantLib as ql
#출처 : Antifragile Domain, by 김성진 퀀트

In [18]:
#평가일 정의
valuationdate = ql.Date(14,6,2019)

In [19]:
ql.Settings.instance().evaluationDate = valuationdate
#평가일 알려주기

#한국 날짜
calender = ql.SouthKorea()

In [20]:
#영업일수 계산(ACT/ACT방식)
dayCount = ql.ActualActual(ql.ActualActual.ISDA)

In [21]:
#Simple quote : 단순히 시장의 데이터를 담고 있는 객체이다. 보통 주가, 금리, 배당률, 변동성 같은 시장 데이터들을 담는 데에 쓰인다.

underlying_qt = ql.SimpleQuote(270.48) # 기초 가격
dividened_qt = ql.SimpleQuote(0.0) # Dividend Yield, 배당 수익률
riskfreerate_qt = ql.SimpleQuote(0.01) #무이자 수익률
volatility_qt = ql.SimpleQuote(0.13) #변동성


In [22]:
# Quote Handle Objects

u_qhd = ql.QuoteHandle(underlying_qt)
q_qhd = ql.QuoteHandle(dividened_qt)
r_qhd = ql.QuoteHandle(riskfreerate_qt)
v_qhd = ql.QuoteHandle(volatility_qt)

In [23]:
# Term-Structure Objects
#기간 구조가 필요한 녀석들을  Term-Structure 객체(시계열인듯 하다)로 변환시켜주는 작업

In [24]:
r_ts = ql.FlatForward(valuationdate, r_qhd, dayCount)
d_ts = ql.FlatForward(valuationdate, q_qhd, dayCount)
v_ts = ql.BlackConstantVol(valuationdate, calender, v_qhd, dayCount)

In [25]:
#Term-Structure Handle Objects

r_thd = ql.YieldTermStructureHandle(r_ts)
d_thd = ql.YieldTermStructureHandle(d_ts)
v_thd = ql.BlackVolTermStructureHandle(v_ts)

In [26]:
# Process & Pricing Engine
process = ql.BlackScholesMertonProcess(u_qhd, d_thd, r_thd, v_thd)
engine = ql.AnalyticEuropeanEngine(process)

In [27]:
#Option Objects 만기가 2023/12/12, 행사가가 272인 유러피안 콜옵션

option_type = ql.Option.Call
strikePrice = 272
expiryDate = ql.Date(12,12,2023)
exercise = ql.EuropeanExercise(expiryDate)
payoff = ql.PlainVanillaPayoff(option_type, strikePrice)
option = ql.VanillaOption(payoff, exercise)

In [28]:
#Pricing
option.setPricingEngine(engine)

In [29]:
#Price & Greeks Results

print('Option Premium = ', round(option.NPV(),2))
print('Option Delta = ', round(option.delta(),4))
print('Option Gamma = ', round(option.gamma(),4))
print('Option Theta = ', round(option.thetaPerDay(),4))
print('Option Vega = ', round(option.vega()/100,4))
print('Option Rho = ', round(option.rho()/100,4))


Option Premium =  34.59
Option Delta =  0.6105
Option Gamma =  0.0051
Option Theta =  -0.0123
Option Vega =  2.1997
Option Rho =  5.8687
